In [1]:
from glob import glob
import os
import fitz  # PyMuPDF 라이브러리
import pdfplumber
import sys
import re
import pandas as pd
from tqdm import tqdm
import traceback


c:\Users\choi seung un\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\choi seung un\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
c:\Users\choi seung un\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [53]:
def find_sentences(text, start_words, end_words=['\n']):
    for start_word in start_words:
        for end_word in end_words:
            pattern = re.compile(f'{start_word}.*?\{end_word}', re.DOTALL)
            matches = pattern.findall(text)
            if len(matches) > 0:
                break
        if len(matches) > 0:
            break
    result = matches[0].replace(start_word, "")
    result = result.replace(end_word, "")
    return result

### use fitz
def read_pdf_use_fitz(file_path, page_number=0):
    doc = fitz.open(file_path)
    page = doc[page_number]
    texts = page.get_text()
    prod_name = find_sentences(texts, ["상품명:", "상품명 :", "상 품 명", "상품명"])
    prod_feat = find_sentences(texts, ["상품특징", "특징"], ["거래조건", "거래 조건"])
    prod_name = prod_name.replace(":", "")
    prod_feat = prod_feat.replace(":", "")
    prod_feat = prod_feat.replace("\n", " ")
    prod_feat = prod_feat.replace("2", "")
    doc.close()
    return prod_name.strip(), prod_feat.strip()


def pdf_to_dataframe(pdf_path):
    page_number = 0
    with pdfplumber.open(pdf_path) as pdf:
        page = pdf.pages[page_number]
        table = list()
        for i in page.extract_tables()[0]:
            table.append([j for j in i if j not in ["", None]] )
        table = [i for i in table if len(i) != 0]
        table = [[i[0], " ".join(i[1:])] if len(i) > 2 else i for i in table]
        table = [[None, i[0]] if len(i) == 1 else i for i in table]
        
        # # print("page_number", page_number+1, len(table))

        df = pd.DataFrame(table)
        # 첫 번째 행을 컬럼으로 사용
        df.columns = df.iloc[0]
        # 첫 번째 행은 더 이상 필요 없으므로 제거
        df = df[1:]
        columns = df.columns
        page_number += 1
        
        while True:
            try:
                page = pdf.pages[page_number]
            except IndexError:
                break
            if len(page.extract_tables()) != 0:
                table_1 = list() 
                for i in page.extract_tables()[0]:
                    table_1.append([j for j in i if j not in ["", None]] )
                table_1 = [i for i in table_1 if len(i) != 0]
                table_1 = [[i[0], " ".join(i[1:])] if len(i) > 2 else i for i in table_1]
                table_1 = [[None, i[0]] if len(i) == 1 else i for i in table_1]
            
                # print("page_number", page_number+1, len(table_1))

                etc = pd.DataFrame(table_1, columns=columns)
                if etc.at[0, columns[1]] == None:
                    etc.at[0, columns[1]] = etc.at[0, columns[0]]
                    etc.at[0, columns[0]] = None
                df = pd.concat([df, etc], axis=0)
            page_number += 1
            if page_number >= len(pdf.pages):
                # print(f"End of the document. Processed {page_number} pages.")
                break
        
        ### 중간 구분에 괄호로만 있는 행이 있어서
        df[columns[0]] = df[columns[0]].str.replace("\n", "")
        df[columns[0]] = df[columns[0]].str.replace(" ", "")
        df[columns[0]] = df[columns[0]].astype(str).apply(lambda x: re.sub(r'\(.*', '', x))
        pattern = re.compile(re.escape("혜택") + '.*?' + re.escape("."))
        df[columns[0]] = df[columns[0]].astype(str).apply(lambda x: re.sub(pattern, '', x))
        
        df.reset_index(drop=True, inplace=True)
        # null인 행들을 순회하면서 이전 행의 '내용'에 추가하고 해당 행 삭제
        null_rows = df[(df[columns[0]].isin(["", "None"])) | (df[columns[0]].isna())]
        for index, row in null_rows.iterrows():
            prev_index = index - 1
            if prev_index >= 0:
                while True:
                    try:
                        df.at[prev_index, columns[1]] += f"\n{row[columns[1]]}"
                        break
                    except KeyError:
                        prev_index -= 1
                        continue
            df.drop(index, inplace=True)
        df.reset_index(drop=True, inplace=True)
        
        # null인 행들을 순회하면서 이전 행의 '구분'에 추가하고 해당 행 삭제
        null_rows = df[(df[columns[1]].isin(["", "None"])) | (df[columns[1]].isna())]
        
        for index, row in null_rows.iterrows():
            prev_index = index - 1
            if prev_index >= 0:
                while True:
                    try:
                        df.at[prev_index, columns[0]] += f"\n{row[columns[0]]}"
                        break
                    except KeyError:
                        prev_index -= 1
                        continue
            df.drop(index, inplace=True)
        df.reset_index(drop=True, inplace=True)    

        def change_spec(text):
            try:
                text = text.strip()
                if text.startswith("-"):
                    text = text.replace("-", "●")
            except AttributeError:
                pass
            return text
        df[columns[0]] = df[columns[0]].apply(lambda x: x.strip()[1:] if x.strip().startswith("-") else x.strip())
        df[columns[1]] = df[columns[1]].apply(lambda x: change_spec(x))
        df.reset_index(drop=True, inplace=True)

        ## 구분에 중복되는 것은 병합해서 1개의 행으로 만듦
        # df = df.groupby(df.columns[0])[df.columns[1]].agg(lambda x: '\n'.join(x)).reset_index()
        
        df.drop(df[df[columns[0]].isin(["0.1", "구분", "용어"])].index, inplace=True, axis=0)
        df.reset_index(drop=True, inplace=True)
        
    return df


In [54]:
pdf_path = "../data/은행상품/기업은행/(심의완료)[상품설명서]IBK스마트적립식중금채.pdf"
# page_number = 0
print(read_pdf_use_fitz(pdf_path))
pdf_to_dataframe(pdf_path)




('IBK스마트적립식중금채', '중소기업은행이 중소기업에 대한 투자와 중장기 대출재원의 조달을 목적으로 발행하는 채권    구  분 정  의 단리채 표면이율을 단리로 계산하여 만기에 원금과 이자를 동시에 지급하는 채권    ※ 월정액 적립식(정액형)    납입정일을 지정하지 않고 월중에 원하는 날짜에 입금, 입금일부터 이자부리, 만기이연  불가 등 자유적립식 상품의 특성도 있으나, 신규금액으로 월적립금액이 정해지고 매월  월적립금 납입횟수 제한 등은 정기적립식과 유사한 적립형태')


,None,"IBK기업은행 준법감시인 심의필 제2023-4732호(2023.10.06.) 유효기간 : 2024.10.05.\n「IBK스마트적립식중금채」상품설명서\n이 설명서는 금융소비자의 권익 보호 및 금융상품에 대한 이해 증진을 위하여「금융소비자\n보호에 관한 법률」및 관련 규정에 의거, 은행의 내부통제절차를 거쳐 금융상품의 주요내용을\n쉽게 이해할 수 있도록 작성한 자료입니다.\n설명내용을 제대로 이해하지 못하였음에도 불구하고 설명을 이해했다는 서명을 하거나 녹취\n기록을 남기시는 경우, 추후 해당 내용과 관련한 권리구제가 어려울 수 있습니다.\n인터넷뱅킹 등 비대면채널을 통해 가입하거나 통장 미발행 요청 시에는 통장 또는 증서가\n교부되지 않습니다.\n이 설명서는 은행이용자의 상품에 대한 이해를 돕기 위한 참고자료이며, 실제 계약은 IBK스마트\n적립식중금채약관, 중소기업금융채권등록필증(통장)약관, 예금거래기본약관, 적립식예금약관을 적용\n합니다. 계약을 신청하는 경우 약관이, 계약을 체결하는 경우 계약서류가 교부됩니다.\n1 상품 개요 및 특징\n⋅상 품 명 : IBK스마트적립식중금채\n⋅상품특징 : 중소기업은행이 중소기업에 대한 투자와 중장기 대출재원의 조달을 목적으로 발행하는 채권\n구 분 정 의\n단리채 표면이율을 단리로 계산하여 만기에 원금과 이자를 동시에 지급하는 채권\n※ 월정액 적립식(정액형)\n납입정일을 지정하지 않고 월중에 원하는 날짜에 입금, 입금일부터 이자부리, 만기이연\n불가 등 자유적립식 상품의 특성도 있으나, 신규금액으로 월적립금액이 정해지고 매월\n월적립금 납입횟수 제한 등은 정기적립식과 유사한 적립형태\n2 거래 조건\n☞ 금리 등 아래의 내용은 고객의 이해를 돕기 위하여 계약 전에 제공되는 자료로 실제 계약\n내용은 수령하시는 통장 또는 비대면채널의 계좌상세조회에 따릅니다.\n(2023.10.06일 현재)\n구 분 내 용\n가입대상 • 개인고객 / 기업고객 (단, 외국인 비거주자, 대기업 및 기관 제외)\n상품유형 • 중소기업금융채권(적립식채권)\n• 신규 : 영업점, 인터넷뱅킹, 모바일뱅킹\n거래방법\n• 해지 : 영업점, 인터넷뱅킹, 모바일뱅킹\n가입금액 • 최저 1만원 이상 (1만원 단위 적립 가능)\n• 최대 2회 입금 가능\n월 적립한도 • 500만원 월 적립횟수\n(단, 계약월수에 해당하는 횟수까지 적립 가능)\n계약기간 • 6개월 이상 3년 이하 (월단위로 가입 가능)"
0,이자지급시기,• 만기일시지급식
1,원금및이자지급제한,"• 계좌에 압류, 가압류, 질권설정 등이 등록될 경우 원금 및 이자지급 제한\n• ..."
2,기본이자율,• 신규일 당시 영업점에 게시한 IBK스마트적립식중금채의 계약기간별 고시금리 적용\...
3,우대이자율,• 계약기간 중 다음의 요건을 충족하고 만기 해지하는 경우 해당 우대금리 제공\n...
4,만기이자산출근거,• 입금액마다 입금일부터 만기일 전일까지의 기간에 대하여 약정이율로 계산한 이자를 ...
5,만기후이자율,• 만기일 당시 영업점에 게시한 IBK스마트적립식중금채의 만기후이율 적용\n․ 만기...
6,중도해지이자율,• 중도해지이자율 = 가입일 당시 계약기간별 고시금리 × 인정률\n납입기간 10%이...
7,중도해지이자산출근거,• 입금액마다 입금일부터 중도해지일 전일까지의 기간에 대하여\n예치기간별 중도해지이...
8,예금자보호여부,비해당 중소기업금융채권은 예금자보호법에 따라 예금보험공사가\n보호하지 않습니다.
9,해지시불이익,• 만기 전 해지 시 중도해지이자 적용 등


In [57]:
df = pd.DataFrame()
paths = "../data/은행상품/기업은행/*.pdf"
error_log_path = "./error.txt"
error_f = open(error_log_path, "w", encoding="utf-8-sig")

for i, path in enumerate(tqdm(glob(paths))):
# for i, path in enumerate(glob(paths)):
    
    # print("start", os.path.basename(path))
    try:
        prod_name, prod_feat = read_pdf_use_fitz(path)
        etc = pdf_to_dataframe(path)
        prod_name_index = len(etc)+1
        etc.at[prod_name_index, etc.columns[0]] = "상품명"
        etc.at[prod_name_index, etc.columns[1]] = prod_name
        prod_feat_index = len(etc)+1
        etc.at[prod_feat_index, etc.columns[0]] = "상품특징"
        etc.at[prod_feat_index, etc.columns[1]] = prod_feat
        etc.set_index(etc.columns[0], inplace=True)
        etc = etc.T.reset_index(drop=True)
        error_f.write(os.path.basename(path)+"\n")
        error_f.write(", ".join(etc.columns)+"\n")
        
        df = pd.concat([df, etc], axis=0)
        df.reset_index(drop=True, inplace=True)
        
    except Exception as e:
        error_f.write(os.path.basename(path)+"\n")
        traceback.print_exc(file=error_f)  # traceback를 파일에 직접 기록
        pass
error_f.close()

# 변경하고자 하는 특정 컬럼의 이름 설정
target_column = ["상품명", "상품특징"]

# 특정 컬럼의 위치를 변경
columns_except_target = [col for col in df.columns if col not in target_column]
target_column.extend(columns_except_target)
df = df[target_column]

# path = "../data/은행상품/기업.csv"
# df.to_csv(path, encoding="utf-8-sig")

100%|██████████| 107/107 [01:35<00:00,  1.12it/s]


In [53]:
path = "../data/은행상품/기업_backup.csv"
df = pd.read_csv(path, encoding="utf-8-sig", index_col=0)
sorted(df.columns)

['가입금액',
 '가입대상',
 '가입시필요서류',
 '가입자격',
 '가입절차',
 '가입제한',
 '거래방법',
 '거래제한',
 '거래중지',
 '계약기간',
 '계약해지방법',
 '계좌부활',
 '계좌분할',
 '고객별우대',
 '고시금리',
 '그룹별우대이자율',
 '금리변동통지서비스',
 '금리우대',
 '기본이자율',
 '기타',
 '납입금액',
 '납입기간',
 '납입회차',
 '담보대출',
 '담보제공',
 '대상수수료',
 '만기이자산출근거',
 '만기자동재예치',
 '만기후금리',
 '만기후이자율',
 '매칭지원금',
 '면제조건',
 '미션우대이자율',
 '부가서비스',
 '부가혜택',
 '분할신규',
 '분할출금',
 '비과세',
 '사용용도검증시스템',
 '상품구성',
 '상품명',
 '상품유형',
 '상품전환',
 '상품특징',
 '세제혜택',
 '소득공제',
 '수수료면제',
 '수수료면제대상수수료',
 '수수료면제제공조건',
 '신규가입특별면제',
 '신규가입특별우대',
 '신규금액',
 '썸특별이자율',
 '알아두실사항',
 '양도및담보제공',
 '양도및명의변경',
 '연계·제휴서비스내용',
 '연계·제휴서비스변경·종료에대한사전통지',
 '연계·제휴서비스이행책임',
 '연계·제휴서비스제공기간',
 '연계·제휴서비스제공요건',
 '연계가능보험상품',
 '예금자보호여부',
 '예금해지시불이익',
 '외환서비스',
 '외환서비스대상수수료',
 '외환서비스제공조건',
 '우대금리',
 '우대금리제공조건',
 '우대사항',
 '우대이율',
 '우대이자율',
 '우대혜택',
 '원금및이자지급제한',
 '원리금지급방식',
 '월적립한도',
 '월적립횟수',
 '위법계약해지권',
 '은행등록발행',
 '이용제한사항',
 '이자계산방법',
 '이자산출근거',
 '이자지급방식',
 '이자지급시기',
 '입출금식혜택수수료면제',
 '자동이체',
 '자료열람요구권',
 '재가입',
 '재예치',
 '저축금액',
 '저축기간',
 '저축기간별이율',

In [51]:
# df['우대이자율'] = df['우대이자율'].fillna('') + df['우대이자율.1'].fillna('')
# df.drop(['우대이자율.1'], axis=1, inplace=True)

# df['연계·제휴서비스내용'] = df['연계·제휴서비스내용'].fillna('') + df['연계·제휴서비스내용.1'].fillna('')
# df.drop(['연계·제휴서비스내용.1'], axis=1, inplace=True)

# df['계약해지방법'] = df['계약해지방법'].fillna('') + df['계약의해지'].fillna('')
# df.drop(['계약의해지'], axis=1, inplace=True)

# df['우대혜택'] = df['우대혜택'].fillna('') + df['IBK기업은행우대혜택'].fillna('')
# df.drop(['IBK기업은행우대혜택'], axis=1, inplace=True)

# df['연계·제휴서비스내용'] = df['연계·제휴서비스내용'].fillna('') + df['연계·제휴서비스'].fillna('')
# df.drop(['연계·제휴서비스'], axis=1, inplace=True)

# df['휴면예금'] = df['휴면예금'].fillna('') + df['휴면예금및출연'].fillna('')
# df.drop(['휴면예금및출연'], axis=1, inplace=True)

# df['특별우대이자율'] = df['특별우대이자율'].fillna('') + df['군급여특별우대'].fillna('')
# df.drop(['군급여특별우대'], axis=1, inplace=True)


In [52]:
path = "../data/은행상품/기업_backup.csv"
df.to_csv(path, encoding="utf-8-sig")

In [55]:
filtered_df = df[[col for col in df.columns if col in ["연계·제휴서비스내용", "연계·제휴서비스변경·종료에대한사전통지"]]]
# filtered_df = df[[col for col in df.columns if "프리미엄" in col]]
rows_with_values = filtered_df[filtered_df.apply(lambda row: row.count() >= 2, axis=1)]
print(len(rows_with_values))
filtered_df

4


,연계·제휴서비스내용,연계·제휴서비스변경·종료에대한사전통지
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,"▪ 보험 내용 : 상해후유장해, 성폭력위로금, 식중독입원일당\n담보명 보장금액 담보...","• 서면, 우편, 팩스, 전화, 전자우편 등의 방법 중 하나의 방법으로 축소·변경\..."
4,NaN,NaN
...,...,...
101,NaN,NaN
102,NaN,NaN
103,NaN,NaN
104,NaN,NaN
